In [1]:
# DS basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# SKLearn stuff
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier

# pickle
import pickle
import json
import requests

In [2]:
from datetime import datetime
from pandas import Series

import category_encoders as ce

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeClassifier

In [3]:
vg2 = 'https://raw.githubusercontent.com/KevWebb/Video-Game-Predictions/master/vgsales-12-4-2019.csv'
vg2= pd.read_csv(vg2)
vg2.head().T

,0,1,2,3,4
Rank,1,2,3,4,5
Name,Wii Sports,Super Mario Bros.,Mario Kart Wii,PlayerUnknown's Battlegrounds,Wii Sports Resort
basename,wii-sports,super-mario-bros,mario-kart-wii,playerunknowns-battlegrounds,wii-sports-resort
Genre,Sports,Platform,Racing,Shooter,Sports
ESRB_Rating,E,NaN,E,NaN,E
Platform,Wii,NES,Wii,PC,Wii
Publisher,Nintendo,Nintendo,Nintendo,PUBG Corporation,Nintendo
Developer,Nintendo EAD,Nintendo EAD,Nintendo EAD,PUBG Corporation,Nintendo EAD
VGChartz_Score,NaN,NaN,NaN,NaN,NaN
Critic_Score,7.7,10,8.2,NaN,8


In [4]:
vg2.dtypes

Rank                int64
Name               object
basename           object
Genre              object
ESRB_Rating        object
Platform           object
Publisher          object
Developer          object
VGChartz_Score    float64
Critic_Score      float64
User_Score        float64
Total_Shipped     float64
Global_Sales      float64
NA_Sales          float64
PAL_Sales         float64
JP_Sales          float64
Other_Sales       float64
Year              float64
Last_Update        object
url                object
status              int64
Vgchartzscore     float64
img_url            object
dtype: object

In [5]:
vg= vg2[['Name','Platform','Genre','Publisher','Developer','Year',
               'Critic_Score','ESRB_Rating','Global_Sales']]
vg= vg.dropna().reset_index(drop=True)

In [6]:
vg.head()

,Name,Platform,Genre,Publisher,Developer,Year,Critic_Score,ESRB_Rating,Global_Sales
0,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,2013.0,9.4,M,20.32
1,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,2014.0,9.7,M,19.39
2,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,2002.0,9.6,M,16.15
3,Call of Duty: Modern Warfare 3,X360,Shooter,Activision,Infinity Ward,2011.0,8.7,M,14.82
4,Call of Duty: Black Ops,X360,Shooter,Activision,Treyarch,2010.0,8.8,M,14.74


In [7]:
def hit(sales):
    if sales >= 1:
        return 1
    else:
        return 0
vg['Best_Seller'] = vg['Global_Sales'].apply(lambda x: hit(x))

In [8]:
vg= vg.drop(columns= 'Global_Sales')

In [9]:
test = vg[vg['Year'] >= 2018]
vg1= vg[vg['Year'] < 2018]
val = vg1[vg1['Year'] >= 2013]
train = vg[vg['Year'] < 2013]

In [10]:
target = 'Best_Seller'


# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 1000].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [11]:
X_train= train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),  
   RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))


# Fit on train, score on val
model = pipeline.fit(X_train, y_train)
y_score = pipeline.score(X_val, y_val)
y_pred = pipeline.predict(X_val)
print('Validation Accuracy', y_pred)

Validation Accuracy [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1
 0 1 0 1 1 1 0 1 1 1 0 0 0 0 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 0 0 0 1 0 1 1 0
 1 1 0 1 1 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0
 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0
 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]


In [15]:
print(y_score)

0.778125


In [16]:
y_proba = pipeline.predict_proba(X_val)
print(y_proba)

[[0.13       0.87      ]
 [0.12       0.88      ]
 [0.35       0.65      ]
 [0.28       0.72      ]
 [0.37       0.63      ]
 [0.34       0.66      ]
 [0.27       0.73      ]
 [0.16       0.84      ]
 [0.33       0.67      ]
 [0.48       0.52      ]
 [0.49       0.51      ]
 [0.22       0.78      ]
 [0.3        0.7       ]
 [0.47       0.53      ]
 [0.12       0.88      ]
 [0.31       0.69      ]
 [0.3        0.7       ]
 [0.35       0.65      ]
 [0.77       0.23      ]
 [1.         0.        ]
 [0.91       0.09      ]
 [0.92       0.08      ]
 [0.36       0.64      ]
 [0.45       0.55      ]
 [0.41       0.59      ]
 [0.84       0.16      ]
 [0.3        0.7       ]
 [0.5        0.5       ]
 [0.3        0.7       ]
 [0.21       0.79      ]
 [0.59       0.41      ]
 [0.29       0.71      ]
 [0.12       0.88      ]
 [0.25       0.75      ]
 [0.14       0.86      ]
 [0.14       0.86      ]
 [0.32       0.68      ]
 [0.52       0.48      ]
 [0.46       0.54      ]
 [0.63       0.37      ]


In [21]:
vg.head().T

,0,1,2,3,4
Name,Grand Theft Auto V,Grand Theft Auto V,Grand Theft Auto: Vice City,Call of Duty: Modern Warfare 3,Call of Duty: Black Ops
Platform,PS3,PS4,PS2,X360,X360
Genre,Action,Action,Action,Shooter,Shooter
Publisher,Rockstar Games,Rockstar Games,Rockstar Games,Activision,Activision
Developer,Rockstar North,Rockstar North,Rockstar North,Infinity Ward,Treyarch
Year,2013,2014,2002,2011,2010
Critic_Score,9.4,9.7,9.6,8.7,8.8
ESRB_Rating,M,M,M,M,M
Best_Seller,1,1,1,1,1


In [17]:
# pickle
# wb = write binary, rb = read binary
pickle.dump(model, open('vgs_model.pkl', 'wb'))

In [18]:
vgsmodel = pickle.load(open('vgs_model.pkl', 'rb'))

In [22]:
url = 'http://localhost:9000/api'


In [27]:
data = test.to_json()
#json.dumps({'Platform':PS4,'Genre': Action,'Publisher': Rockstar Games,'Developer':Rockstar North,'Year':2011,
 #              'Critic_Score':9.2,'ESRB_Rating':M})

In [28]:
send = requests.post(url, data)

ConnectionError: HTTPConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /api (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001ACB70AC048>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
print(send.json())